In [1]:
import datetime
import re
import pandas as pd
import requests
import vk

from bs4 import BeautifulSoup

pd.set_option('max_colwidth', 100)

### Задание 1

In [2]:
FIRST_PAGE = 1
LAST_PAGE = 2
BASE_URL = "https://habr.com/ru/all/"
KEYWORDS = ["python", "парсинг"]


def parser():
    data = pd.DataFrame()

    for i in range(FIRST_PAGE, LAST_PAGE + 1):
        page_articles = get_page_articles(BASE_URL + "page" + str(i))
        for article in page_articles:
            data = pd.concat([data, pd.DataFrame([article])], ignore_index=True)
        print(f"Закончен парсинг страницы №: {i}")

    return data

#### Обязательная часть

In [3]:
def get_page_articles(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    items = soup.find_all("article", class_="post")

    articles = []
    for item in items:
        preview = item.find("div", class_="post__text").get_text(strip=True)

        if re.findall("|".join(KEYWORDS), preview, flags=re.IGNORECASE):
            articles.append({
                "time": item.find("span", class_="post__time").get_text(),
                "title": item.find("a", class_="post__title_link").get_text(strip=True),
                "link": item.find("a", class_="post__title_link").get("href")
            })

    return articles

    
display(parser())

Закончен парсинг страницы №: 1
Закончен парсинг страницы №: 2


,time,title,link
0,сегодня в 18:23,Распознаем речь с помощью IBM Speech-to-Text API,https://habr.com/ru/company/otus/blog/542904/
1,сегодня в 13:03,Быстрое сравнение double,https://habr.com/ru/post/542836/


#### Дополнительная часть

In [4]:
def get_page_articles(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, "html.parser")
    items = soup.find_all("article", class_="post")

    articles = []
    for item in items:
        preview = item.find("div", class_="post__text").get_text(strip=True)
        link = item.find("a", class_="post__title_link").get("href")
        text = get_article_text(link)

        if re.findall("|".join(KEYWORDS), preview, flags=re.IGNORECASE) or text:
            articles.append({
                "time": item.find("span", class_="post__time").get_text(),
                "title": item.find("a", class_="post__title_link").get_text(strip=True),
                "link": link,
                "text": text
            })

    return articles


def get_article_text(link):
    page = requests.get(link)
    soup = BeautifulSoup(page.text, "html.parser")
    article_text = soup.find("div", id="post-content-body").get_text(strip=True)

    if re.findall("|".join(KEYWORDS), article_text, flags=re.IGNORECASE):
        return article_text
    
    
display(parser())

Закончен парсинг страницы №: 1
Закончен парсинг страницы №: 2


,time,title,link,text
0,сегодня в 19:17,"Рефакторинг пет проекта: докеризация, метрики, тесты",https://habr.com/ru/post/542978/,"Всем привет, я php разработчик. Я хочу поделиться историей, как я рефакторил один из своих телег..."
1,сегодня в 18:23,Распознаем речь с помощью IBM Speech-to-Text API,https://habr.com/ru/company/otus/blog/542904/,"Привет, Хабр. В рамках курса""Machine Learning. Advanced""подготовили для вас перевод интересного ..."
2,сегодня в 18:17,"Как образовательный коптер помогает научиться программировать на Python, и что не так с Lua",https://habr.com/ru/post/542960/,Почти 4 года прошло с выпуска первойстатьиоб учебном квадрокоптере Геоскан Пионер. За это время ...
3,сегодня в 17:17,Делаем детектор настроения собаки с использованием Google Cloud и Raspberry Pi на основе Arm,https://habr.com/ru/company/skillfactory/blog/542888/,"В главной роли на кдпв — Боб, песель ДенискинаСегодня смартфоны, умные часы и фитнес-трекеры ест..."
4,сегодня в 16:17,Российский селлер сравнил между собой сервисы внешней аналитики маркетплейсов. Кому из них довер...,https://habr.com/ru/post/542890/,"На днях, надругойнебезызвестной вам площадке появился большой сравнительный анализ сервисов внеш..."
5,сегодня в 15:35,Как преобразовать текст в речь с использованием Google Tesseract и Arm NN на Raspberry Pi,https://habr.com/ru/company/skillfactory/blog/541764/,"Привет, Хабр! Сегодня специально к старту нового потока курса поMaсhine Learningделимся с вами п..."
6,сегодня в 13:03,Быстрое сравнение double,https://habr.com/ru/post/542836/,"Вчера здесь вышластатьяо быстром парсинге double, я зашёл во блог к её автору, и нашёл тамещё од..."
7,сегодня в 12:48,Разведопрос: из жизни программиста Группы «М.Видео-Эльдорадо»,https://habr.com/ru/company/mvideo/blog/542826/,Мы продолжаем рассказывать вам о жизни и ключевых проектах ребят из нашей внутренней команды раз...


### Задание 2 - Обязательная часть

In [5]:
url = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:85.0) Gecko/20100101 Firefox/85.0",
    "Vaar-Header-App-Product": "hackcheck-web-avast",
    "Vaar-Version": "0"
}

payload = {
    "emailAddresses": ["xxx@ya.ru", "yyy@ya.ru"]
}

data_breaches = pd.DataFrame()
data_summary = pd.DataFrame()

req = requests.post(url, headers=headers, json=payload)

for value in req.json()["breaches"].values():
    row = {
        "breach_id": value["breachId"],
        "publish_date": pd.to_datetime(value["publishDate"]).strftime("%Y-%m-%d"),
        "site": value["site"],
        "description": value["description"]
    }
    data_breaches = pd.concat([data_breaches, pd.DataFrame([row])], ignore_index=True)

for mail, breaches in req.json()["summary"].items():
    for breach_id in breaches["breaches"]:
        row = {"breach_id": breach_id, "mail": mail}
        data_summary = pd.concat([data_summary, pd.DataFrame([row])], ignore_index=True)

joined = data_summary.merge(data_breaches, on="breach_id")

display(joined[["mail", "publish_date", "site", "description"]])

,mail,publish_date,site,description
0,xxx@ya.ru,2016-11-07,ir.netease.com,"In October 2015, the Chinese internet and gaming company NetEase suffered a data breach that lea..."
1,xxx@ya.ru,2020-07-30,hurb.com,"In March 2019, the Brazilian travel agency Herb was allegedly breached. The stolen data contains..."
2,xxx@ya.ru,2016-11-01,qip.ru,"In 2011, Russian instant messaging service provider QIP was allegedly breached. The leaked data ..."
3,xxx@ya.ru,2019-12-05,wowprogress.com,"At an unconfirmed date, the World of Warcraft fan site WowProgress was allegedly breached. The s..."
4,xxx@ya.ru,2017-11-04,myheritage.com,"In October 2017, a customer database belonging to online genealogy platform MyHeritage was breac..."
5,xxx@ya.ru,2020-02-06,flyffpenya-shop.com,"In 2015, online gaming shop Flyffpenya Shop was allegedly breached. The stolen data contains pas..."
6,xxx@ya.ru,2020-02-06,cityofvillains.com,"In January 2015, the online role-playing game City of Villains' database was allegedly breached...."
7,xxx@ya.ru,2016-10-24,dropbox.com,Cloud storage company Dropbox suffered a major data breach in 2012. Criminals leveraged stolen e...
8,xxx@ya.ru,2016-10-29,vk.com,Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million...
9,yyy@ya.ru,2016-10-29,vk.com,Popular Russian social networking platform VKontakte was breached in late 2012. Over 100 million...


### Задание 2 - Дополнительная часть

In [6]:
session = vk.Session(token)
vkapi = vk.API(session)

posts = vkapi.wall.get(domain='netology', count=50, v=5.65)

wall_posts = pd.DataFrame()
for post in posts["items"]:
    row = {
        "date": datetime.datetime.fromtimestamp(post["date"]).strftime("%Y-%m-%d"),
        "text": " ".join(post["text"].replace("\n", "").split())}
    wall_posts = pd.concat([wall_posts, pd.DataFrame([row])], ignore_index=True)

display(wall_posts)

,date,text
0,2021-02-17,"Представьте, что вы трудитесь в секретной организации и никто не должен знать, где вы работаете ..."
1,2021-02-17,📆 22 февраля стартует бесплатный курс «Как стать продакт- или проджект-менеджером» Менеджеры нуж...
2,2021-02-16,Однажды в вашей карьере настанет момент делегирования части обязанностей. Чтобы в итоге не перед...
3,2021-02-16,На прошлой неделе в мире диджитала произошло много событий: от массового ухода пользователей в Т...
4,2021-02-15,"Теперь можно не только слушать подкасты, но и свободно принимать в них участие. В Медиа разбирае..."
5,2021-02-15,"Направлений в дизайне много: можно делать рекламные баннеры, проектировать сайты или делать граф..."
6,2021-02-14,"«Жизнь как коробка шоколадных конфет: никогда не знаешь, какая начинка попадётся» 🍬 В честь Дня ..."
7,2021-02-13,"*партнёрский пост* 26-28 февраля пройдёт онлайн-хакатон «SberCode» Разработчики, дизайнеры, прод..."
8,2021-02-13,"Собрали статьи для начинающих Java-разработчиков: разбираемся, почему Java — хороший выбор для н..."
9,2021-02-12,"В эфире #пятничнаябеседка. Здесь мы обсуждаем, рассуждаем, спорим и аргументируем. Не зря говоря..."
